In [22]:
from ase.lattice.cubic import SimpleCubicFactory, BodyCenteredCubicFactory

from ase.lattice.tetragonal import CenteredTetragonalFactory

class Mn2AuFactory(CenteredTetragonalFactory):
    "A factory for creating Mn2Au lattices."
    bravais_basis = [[0.0, 0.0,  0.333333], [0.0 , 0.0, 0.666666], [0, 0, 0]]
    element_basis = (0, 0, 1)

Mn2Au = Mn2AuFactory()

alat = 3.1445
factor = 2.5658

cubic_unit_cell = Mn2Au(["Mn", "Au"], latticeconstant=[alat, factor * alat]) 

print(cubic_unit_cell)
cubic_unit_cell.wrap()
cubic_unit_cell.write("Mn2Au_cubic.xyz")

Lattice(symbols='Mn2AuMn2Au', pbc=True, cell=[3.1445, 3.1445, 8.0681581])


In [23]:
cubic_unit_cell.pbc


array([ True,  True,  True])

In [61]:
1.0 / cubic_unit_cell.cell.lengths() / 0.03618256

array([8.7891952 , 8.7891952 , 3.42551844])

In [24]:
cubic_unit_cell.positions

array([[ 1.20737065e-17, -1.20737065e-17,  2.68938334e+00],
       [ 2.41474131e-17, -2.41474131e-17,  5.37876669e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.57225000e+00,  1.57225000e+00,  6.72346239e+00],
       [ 1.57225000e+00,  1.57225000e+00,  1.34468764e+00],
       [ 1.57225000e+00,  1.57225000e+00,  4.03407905e+00]])

In [3]:
# The lattice units from the paper
6.29 / 1.8897259886

3.3285248961728757

In [4]:
# c/a from the paper
16.142 / 6.291

2.5658877761882053

In [49]:
from ase import Atoms

def make_primitive_Mn2Au_cell(alat, factor):
    
    cell = [[-alat / 2.0, alat / 2.0, alat * factor / 2.0],
            [alat / 2.0, -alat / 2.0, alat * factor / 2.0],
            [alat / 2.0, alat / 2.0, -alat * factor / 2.0]
           ]
    
    Au = Atoms("Au", scaled_positions=[[0.0, 0.0, 0.0]], cell=cell, pbc=True)
    Mn2 = Atoms("Mn2", scaled_positions=[[0.333, 0.333, 0.0], [0.6666, 0.666, 0.0]], cell=cell, pbc=True)

    return Au + Mn2


In [50]:
primitive_unit_cell = make_primitive_Mn2Au_cell(alat, factor)
print(primitive_unit_cell)

Atoms(symbols='AuMn2', pbc=True, cell=[[-1.57225, 1.57225, 4.03407905], [1.57225, -1.57225, 4.03407905], [1.57225, 1.57225, -4.03407905]])


In [51]:
primitive_unit_cell.positions

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  2.68669665e+00],
       [-9.43350000e-04,  9.43350000e-04,  5.37581374e+00]])

In [59]:
(1.0 / primitive_unit_cell.cell.lengths()) / 6.0

array([0.03618256, 0.03618256, 0.03618256])

In [52]:
primitive_unit_cell.get_all_distances()

array([[0.        , 2.68669665, 5.37581391],
       [2.68669665, 0.        , 2.68911743],
       [5.37581391, 2.68911743, 0.        ]])

In [ ]:
cubo

In [53]:
primitive_unit_cell.write("Mn2Au_primitive.xyz")

In [15]:
from ase.calculators.vasp import Vasp

n_par = 10

VaspCalc = Vasp(setups={'W': '_sv'}, # Check pseudopotentials for Mn and Au here
                npar=n_par, kpts=[3,3,3],
                istart=0, # start from scratch
                icharg=2, # default for istart=0
                isif=2, 
                nsim=2,
                prec='Accurate',
                encut=500,
                ediff=1.e-6,
                ediffg=-0.04,
                nelm=100,
                nelmin=5,
                algo="Normal",
                isym=0,
                ismear=1,
                sigma=0.1,
                ispin=2, # spin polarised calculation
                magmom=[0.0, 2.0, -2.0], # a first guess for the Mn2Au
                xc='PBE')

In [21]:
VaspCalc.write_incar(cubic_unit_cell)

ValueError: Expected length of magmom tag to be 6, i.e. 1 value per atom, but got 3

In [ ]:
VaspCalc

In [20]:
primitive_unit_cell

Atoms(symbols='AuMn2', pbc=True, cell=[[-1.57225, 1.57225, 4.03407905], [1.57225, -1.57225, 4.03407905], [1.57225, 1.57225, -4.03407905]])

In [18]:
cubic_unit_cell

Lattice(symbols='Mn2AuMn2Au', pbc=True, cell=[3.1445, 3.1445, 8.0681581])

In [37]:
cubic_unit_cell.get_all_distances()

array([[0.        , 2.68938334, 2.68938334, 4.60627115, 2.5984893 ,
        2.5984893 ],
       [2.68938334, 0.        , 5.37876669, 2.5984893 , 4.60627115,
        2.59848513],
       [2.68938334, 5.37876669, 0.        , 7.08158786, 2.59848513,
        4.60627115],
       [4.60627115, 2.5984893 , 7.08158786, 0.        , 5.37877476,
        2.68938334],
       [2.5984893 , 4.60627115, 2.59848513, 5.37877476, 0.        ,
        2.68939141],
       [2.5984893 , 2.59848513, 4.60627115, 2.68938334, 2.68939141,
        0.        ]])

## The both cells give similar results:

Cubic cell enrgy: -42.01578777
Energy per atom: -7.0026312950000005
Forces
```
[[-1.23000000e-06 -1.16000000e-06 -1.00935569e+00]
 [ 5.30000000e-07 -9.60000000e-07  1.01005875e+00]
 [-5.30000000e-07  2.86000000e-06 -7.73770000e-04]
 [-5.20000000e-07  7.40000000e-07 -1.01074019e+00]
 [-2.00000000e-08 -7.00000000e-07  1.01015928e+00]
 [ 1.78000000e-06 -7.90000000e-07  6.51620000e-04]]
 ```
Stress
```
[-1.42497944e-01 -1.42497746e-01 -1.27948453e-01  2.81429646e-08
  1.66648294e-09  1.42481170e-07]
```
Total magnetic moment
0.0011857
Magnetic moments:
```
[-3.209  3.209 -0.    -3.209  3.209  0.   ]
```
Primitive

-20.99605531
Energy per atom: -6.998685103333333
Forces
```
[[-0.00552589  0.00552338 -0.04020551]
 [-0.00256008  0.00255829 -0.99728084]
 [ 0.00808597 -0.00808167  1.03748635]]
```
Stress
```
[-1.39089114e-01 -1.39089206e-01 -1.28556836e-01  1.46399977e-04
 -1.46068509e-04  8.86930930e-07]
```
Total magnetic moment
0.0018399
Magnetic moments:
```
[ 1.000e-03 -3.188e+00  3.189e+00]
```


## TO DO:

- Energy cut off and k-points convergence tests
- Identify lattice constants and zero pressure and elastic constants
- Apply the straining

## To discuss:
- Importance and relevance of [Dudarev](https://www.vasp.at/wiki/index.php/LDAUTYPE#cite_note-rohrbach:jcp:03-1) approach